In [1]:
import os


In [2]:
%pwd

'/Users/vivek/work/GIT/end-to-end-mlflow/research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'/Users/vivek/work/GIT/end-to-end-mlflow'

### defining the entity

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path
    trained_model_path: Path
    updated_base_model_path: Path
    training_data: Path
    params_epochs: int
    params_batch_size: int
    params_is_augmentation: bool
    params_image_size: list

### adding the configuration manager


In [6]:
from src.cnnClassifier.constants import *
from src.cnnClassifier.utils.common import read_yaml, create_directories
import tensorflow as tf


In [7]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH):

            self.config = read_yaml(config_filepath)
            self.params = read_yaml(params_filepath)

            create_directories([self.config.artifacts_root])
    def get_training_config(self)-> TrainingConfig:
          training = self.config.training
          prepare_base_modle = self.config.prepare_base_model
          params = self.params
          trainng_data = os.path.join(self.config.)
    